# II Konvoliuciniai neuroniniai tinklai

## Temos

<code>**1  : Konvoliucijos erdvės užpildas**</code><br>
<code>**2  : Erdvinis iškritimas ir BatchNorm**</code><br>
<code>**3  : Dažniausios struktūro: VGG16, ResNet**</code><br>
<code>**4  : VGG16 (PyTorch)**</code><br>
<code>**5  : ResNet (PyTorch)**</code><br>
<code>**6  : Max-Pooling pakeitimas konvoliuciniais sluoksniais**</code><br>
<code>**7  : Konvoliucinis sluoksnis vietoje visiškai sujungtų sluoksnių**</code><br>
<code>**8  : Inception, - lygiagrečios konvoliucijos ir pagalbiniai skaičiavimai**</code><br>
<code>**9 : Perkeliamas mokymas (PyTorch)**</code><br>

In [2]:
#TODO apprx: 2:15:00

## Konvoliucijos erdvės užpildas (padding)

Tai mechanizmas kuris leidžia mums kontroliuoti išvesties dydį kartu su `stride` operacija. Kol kas kalbėjomės apie paprastus atvejus kuomet `stride` parametras buvo lygus 1. Tai reiškiasi kad mūsų filtras per įvestį juda po vieną pozicija iš kairės į dešinę atliekant konvoliuciją. Jeigu šis parametras bus lygus 2 tuomet filtras judės per dvi pozicijas ir po konvoliucijos operacijos išvestis bus du maždaug du kartus mažesnė nei įvestis. `Padding` yra atvikrštnis procesas kuris leidžia padaryti kad išvestis būtų didensė. Veikimo principas toks lyg aplinkui įvestį sukuriame papildomą erdvę kurios reikšmės dažniausiai yra lygios 0.

\begin{multline*}
o = \dfrac{i + 2p  - k}{s} + 1
\end{multline*}

 - `o` - išvestis
 - `i` - įvesties dydis
 - `p` - užpildo erdvės dydis
 - `k` - filtor dydis
 - `s` - žingsnio dydis

Mes šią lygtį jau naudojome apskaičiuote kokią bus išvestis atživelgiant į filtro ir žingsnio dydį.